# Prepare SQuAD_tiny Dataset for Assignment 2

This code prepare SQuAD_tiny from the SQuAD dataset. 

# 0. Import libraries

In [3]:
import os
import torch
from datasets import load_dataset
from transformers import T5Tokenizer, T5ForConditionalGeneration, Trainer, TrainingArguments, DataCollatorForSeq2Seq
from rouge_score import rouge_scorer
from tqdm import tqdm

In [4]:
# Set seed for reproducibility
torch.manual_seed(42)

# 1. Load and preprocess SQuAD dataset

In [6]:
# 1. Load and preprocess SQuAD dataset
dataset = load_dataset("squad")

In [7]:
# Take subsets to avoid overload
train_dataset = dataset["train"].select(range(10000))
val_dataset = dataset["validation"].select(range(1000))
test_dataset = dataset["validation"].select(range(1000, 2000))  # No official SQuAD test set

In [8]:
# Load tokenizer and model
tokenizer = T5Tokenizer.from_pretrained("t5-small")
model = T5ForConditionalGeneration.from_pretrained("t5-small")

You are using the default legacy behaviour of the <class 'transformers.models.t5.tokenization_t5.T5Tokenizer'>. This is expected, and simply means that the `legacy` (previous) behavior will be used so nothing changes for you. If you want to use the new behaviour, set `legacy=False`. This should only be set if you understand what it means, and thoroughly read the reason why this was added as explained in https://github.com/huggingface/transformers/pull/24565


In [9]:
# Preprocessing function
def preprocess(example):
    input_text = f"question: {example['question']}  context: {example['context']}"
    target_text = example["answers"]["text"][0]
    input_enc = tokenizer(input_text, padding="max_length", truncation=True, max_length=512)
    target_enc = tokenizer(target_text, padding="max_length", truncation=True, max_length=32)
    input_enc["labels"] = target_enc["input_ids"]
    return input_enc

In [10]:
# Preprocess the datasets
train_enc = train_dataset.map(preprocess, batched=False)
val_enc = val_dataset.map(preprocess, batched=False)
test_enc = test_dataset.map(preprocess, batched=False)

Map:   0%|          | 0/10000 [00:00<?, ? examples/s]

Map:   0%|          | 0/1000 [00:00<?, ? examples/s]

Map:   0%|          | 0/1000 [00:00<?, ? examples/s]

In [25]:
# Prepare 20 data points for qualitative analysis

q_data = test_dataset.select(range(20))

q_data

Dataset({
    features: ['id', 'title', 'context', 'question', 'answers'],
    num_rows: 20
})